In [1]:
import pandas as pd
import numpy as np
import time
import gc

In [2]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train = pd.read_csv(path + 'train_cleaned_final.csv')
test = pd.read_csv(path + 'test_cleaned_final.csv')

# Get Train on Day7 Day8 Day9

In [4]:
df_hour = pd.read_csv(path+'hourdistri.csv', index_col='Unnamed: 0')
index = {}
for day in ['day7', 'day8','day9']:
    index[day] = list(range(df_hour.loc[day,'4start'], df_hour.loc[day,'6end0sec'])) + \
    list(range(df_hour.loc[day,'9start'], df_hour.loc[day,'11end0sec'])) + \
    list(range(df_hour.loc[day,'13start'], df_hour.loc[day,'15end0sec'])) 

# Feature Col

In [13]:
target = 'is_attributed'

feature_countInTrain = [
                    'ip_day_hour_countInTrain',
                    'ip_os_day_hour_countInTrain',
                    'ip_app_day_hour_countInTrain',
                    'ip_app_os_day_hour_countInTrain',
                    'app_day_hour_countInTrain',
                    'ip_device_os_countInTrain',
                    'ip_app_device_os_countInTrain']

feature_countInTest = [
                    'ip_day_hour_countInTest',
                    'ip_os_day_hour_countInTest',
                    'ip_app_day_hour_countInTest',
                    'ip_app_os_day_hour_countInTest',
                    'app_day_hour_countInTest',
                    'ip_device_os_countInTest',
                    'ip_app_device_os_countInTest']

                         

feature_ori = ['app', 'channel', 'device', 'os', 'hour']
feature_ori = []

feature_cols = []
added_feature = []

added_feature.extend(feature_countInTrain)
added_feature.extend(feature_countInTest)
feature_cols.extend(added_feature)
feature_cols.extend(feature_ori)

train_cols = feature_cols.copy()
train_cols.append(target)

In [14]:
feature_cols

['ip_day_hour_countInTrain',
 'ip_os_day_hour_countInTrain',
 'ip_app_day_hour_countInTrain',
 'ip_app_os_day_hour_countInTrain',
 'app_day_hour_countInTrain',
 'ip_device_os_countInTrain',
 'ip_app_device_os_countInTrain',
 'ip_day_hour_countInTest',
 'ip_os_day_hour_countInTest',
 'ip_app_day_hour_countInTest',
 'ip_app_os_day_hour_countInTest',
 'app_day_hour_countInTest',
 'ip_device_os_countInTest',
 'ip_app_device_os_countInTest']

# Define functin

In [15]:
def count(df_history, df_train, cols, target=None):
    """
    Purpose: add a new feature to training df.count the number of records for each feature combination (eg, artist_name_composer)) 
    """
    
    group = get_group(df_train, cols)
    group_all = get_group(df_history, cols)
    
    count_map = group_all.value_counts()
    
    return group.map(count_map).fillna(0)


def countInTrain(df_history, df_train, cols, target=None):
    """
    Purpose: add a new feature to training df.count the number of records for each feature combination (eg, artist_name_composer)) 
    """
    
    group = get_group(df_train, cols)
    group_all = get_group(df_history, cols)
    
    count_map = group_all.value_counts()
    
    return group.map(count_map).fillna(0)


def countInTest(df_history, df_train, cols, target=None):
    """
    Purpose: add a new feature to training df.count the number of records for each feature combination (eg, artist_name_composer)) 
    """
    
    group = get_group(df_train, cols)
    group_all = get_group(df_history, cols)
    
    count_map = group_all.value_counts()
    
    return group.map(count_map).fillna(0)



def col_name(cols, func=None):
    if func is None:
        return '_'.join(cols)
    else:
        return '_'.join(cols) + '_' + func.__name__
    
    



In [16]:
orders = {}
feature_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel',
              'day',
              'hour',]

# feature_col = ['ip', 
#               'app', 
#               'device', 
#               'os', 
#               'channel']
for col in feature_col:
    orders[col] = 10 ** (int(np.log(max(train[col].max(),test[col].max() ) + 1) / np.log(10)) + 1)
def get_group(df, cols):
    """
    define an encoding method which can ganrantee the adding value will be unique.
    eg: artist_name_composer will be a combination of (artist_name,composer) and the encoding will reflect the unqiue combination of those two
    """
    group = df[cols[0]].copy()
    for col in cols[1:]:
        group = group * orders[col] + df[col]
        
    return group

# Feature engineering on Train

In [ ]:
import sys
from itertools import combinations
combine_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel',
              'day',
              'hour',]




for day in ['day7', 'day8', 'day9']:
    counter = 0
    df_train = train.iloc[index[day]].copy()
    print('got train data')
    print('got historical data')
    
    ###########################################################################
    for func in [countInTrain, countInTest]:
                if func.__name__ == count.__name__:
                    df_all = pd.concat([train, test])
                else:
                    try:
                        del df_all
                        gc.collect()
                    except Exception:
                        print('df_all does not exist')
               
                for num_col in [1,2,3,4,5]:
                    for cols in combinations(combine_col, num_col):
                        feature_name = col_name(cols, func=func)
                        if feature_name not in added_feature:
                               continue
                        counter += 1
                        if func.__name__ == count.__name__:
                                print('count function')
                                df_train[feature_name] = func(df_all, df_train, cols, target='is_attributed')

                        elif func.__name__ == countInTrain.__name__:
                                print('count in Train function')
                                df_train[feature_name] = func(train, df_train, cols, target='is_attributed')
                                
                        elif func.__name__ == countInTest.__name__:
                                print('count in Test function')
                                df_train[feature_name] = func(test, df_train, cols, target='is_attributed')

                        else:
                                print('time related function')
                                df_train[feature_name] = func(df_train, df_train, cols, target='is_attributed')
               
                        all_str = 'all {}:   {}   \t\t\t size: {} G.'.format(counter, feature_name, sys.getsizeof(df_train)/ 1024 **3)
                        print(all_str)
                        with open('feature_all.txt', 'w') as text_file:
                            text_file.write(all_str + '\n')

               
               
    save_file_name = '{}_features_CountInTrainTest.csv'.format(day)
    save_file_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/' + save_file_name
    df_train = df_train[train_cols]
    print('------')
    print(df_train.columns.values)
    df_train.to_csv(save_file_path, index=False)
    print(save_file_path)
    print('======================================================')

got train data
got historical data
df_all does not exist
count in Train function
all 1:   ip_device_os_countInTrain   			 size: 2.0376134142279625 G.
count in Train function
all 2:   ip_day_hour_countInTrain   			 size: 2.183157227933407 G.
count in Train function
all 3:   app_day_hour_countInTrain   			 size: 2.328701041638851 G.
count in Train function
all 4:   ip_app_device_os_countInTrain   			 size: 2.4742448553442955 G.
count in Train function
all 5:   ip_app_day_hour_countInTrain   			 size: 2.61978866904974 G.
count in Train function
all 6:   ip_os_day_hour_countInTrain   			 size: 2.765332482755184 G.
count in Train function
all 7:   ip_app_os_day_hour_countInTrain   			 size: 2.9108762964606285 G.
df_all does not exist
count in Test function
all 8:   ip_device_os_countInTest   			 size: 3.056420110166073 G.
count in Test function
all 9:   ip_day_hour_countInTest   			 size: 3.201963923871517 G.
count in Test function
all 10:   app_day_hour_countInTest   			 size: 3.3475077375

# Train on Test

In [ ]:
import sys
from itertools import combinations
combine_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel',
              'day',
              'hour',]



counter = 0
df_train = test.copy()
print('got train data')
print('got historical data')

###########################################################################
for func in [countInTrain, countInTest]:
        if func.__name__ == count.__name__:
            df_all = pd.concat([train, test])
        else:
            try:
                del df_all
                gc.collect()
            except Exception:
                print('df_all does not exist')

        for num_col in [1,2,3,4,5]:
            for cols in combinations(combine_col, num_col):
                feature_name = col_name(cols, func=func)
                if feature_name not in added_feature:
                       continue
                counter += 1
                if func.__name__ == count.__name__:
                        print('count function')
                        df_train[feature_name] = func(df_all, df_train, cols, target='is_attributed')

                elif func.__name__ == countInTrain.__name__:
                        print('count in Train function')
                        df_train[feature_name] = func(train, df_train, cols, target='is_attributed')

                elif func.__name__ == countInTest.__name__:
                        print('count in Test function')
                        df_train[feature_name] = func(test, df_train, cols, target='is_attributed')

                else:
                        print('time related function')
                        df_train[feature_name] = func(df_train, df_train, cols, target='is_attributed')

                all_str = 'all {}:   {}   \t\t\t size: {} G.'.format(counter, feature_name, sys.getsizeof(df_train)/ 1024 **3)
                print(all_str)
                with open('feature_all.txt', 'w') as text_file:
                    text_file.write(all_str + '\n')






save_file_name = '{}_features_CountInTrainTest.csv'.format('test')
save_file_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/' + save_file_name
df_train = df_train[feature_cols]
print(df_train.columns.values)
df_train.to_csv(save_file_path, index=False)
print(save_file_path)

got train data
got historical data
df_all does not exist
count in Train function
all 1:   ip_device_os_countInTrain   			 size: 1.679998941719532 G.
count in Train function
all 2:   ip_day_hour_countInTrain   			 size: 1.8199988454580307 G.
count in Train function
all 3:   app_day_hour_countInTrain   			 size: 1.9599987491965294 G.
count in Train function
all 4:   ip_app_device_os_countInTrain   			 size: 2.099998652935028 G.
count in Train function
all 5:   ip_app_day_hour_countInTrain   			 size: 2.2399985566735268 G.
count in Train function
all 6:   ip_os_day_hour_countInTrain   			 size: 2.3799984604120255 G.
count in Train function
all 7:   ip_app_os_day_hour_countInTrain   			 size: 2.519998364150524 G.
df_all does not exist
count in Test function
all 8:   ip_device_os_countInTest   			 size: 2.659998267889023 G.
count in Test function
all 9:   ip_day_hour_countInTest   			 size: 2.7999981716275215 G.
count in Test function
all 10:   app_day_hour_countInTest   			 size: 2.9399980